In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pro_data

# 2.3.8 失信数据8breakfaith.csv提取特征
    1. 企业失信的个数，BREAKFAITH_CNT
    2. 企业第一个失信数据的日期，BREAKFAITH_FIRST_FIRST_TIME
    3. 企业最后一个失信数据的日期，BREAKFAITH_FIRST_END_TIME
    4. 企业第一个失信数据的到期日期，BREAKFAITH_END_FIRST_TIME
    5. 企业第一个失信数据的日期与其结束日期的差值，BREAKFAITH_FIRST_END_FIRST_DIFF

    5. 企业第一个失信数据的日期与企业注册日期的差值，BREAKFAITH_FIRST_RGYEAR_DIFF
    6. 企业最后一个失信数据的日期与企业注册日期的差值，BREAKFAITH_END_RGYEAR_DIFF

    7. 企业第一个失信数据的到期日期与企业注册日期的差值，BREAKFAITH_END_RGYEAR_DIFF

    8. 企业第一个失信数据的日期与企业第一次变更的差值，BREAKFAITH_FIRST_CHANGE_FIRST_DIFF
    9. 企业最后一个失信数据的日期与企业第一次变更的差值，BREAKFAITH_END_CHANGE_FIRST_DIFF
    
    10. 企业第一个失信数据的日期与企业最后一次变更的差值，BREAKFAITH_FIRST_CHANGE_END_DIFF
    11. 企业最后一个失信数据的日期与企业最后一次变更的差值，BREAKFAITH_END_CHANGE_END_DIFF
    
    ---------------------------------------------------------------------------------------
    10. 企业平均每几个月的失信一次， BREAKFAITH_PRE_MONTH_CNT = BREAKFAITH_END_RGYEAR_DIFF / BREAKFAITH_CNT
    
    11. 企业失信的个数占所有失信个数个平均值的比例，BREAKFAITH_CNT_ALL_RATE
    
    12. 企业对应的大类HY的平均失信数据的个数，BREAKFAITH_HY_CNT_AVG
    13. 企业对应大类HY的平均失信个数占所有失信平均个数的比例，BREAKFAITH_HY_CNT_ALL_RATE
    14. 企业失信的个数占其对应的大类HY的失信平均值的比例，BREAKFAITH_CNT_HY_RATE

    

In [5]:
df_all = pd.read_csv("../data/alldata/df_data1234567.csv")
df_breakfaith = pd.read_csv("../data/public/8breakfaith.csv")

In [6]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255130 entries, 0 to 255129
Columns: 178 entries, EID to LAWSUIT_LAWAMOUNT_MEAN_HY_RATE
dtypes: float64(113), int64(48), object(17)
memory usage: 346.5+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,LAWSUIT_FIRST_CHANGE_END_TIME_DIFF,LAWSUIT_PRE_MONTH_CNT,LAWSUIT_CNT_ALL_RATE,LAWSUIT_LAWAMOUNT_MEAN_ALL_RATE_x,LAWSUIT_HY_CNT_AVG,LAWSUIT_HY_CNT_ALL_RATE,LAWSUIT_LAWAMOUNT_MEAN_HY_AVG,LAWSUIT_LAWAMOUNT_MEAN_ALL_RATE_y,LAWSUIT_CNT_HY_RATE,LAWSUIT_LAWAMOUNT_MEAN_HY_RATE
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,34,inf,0.0,0.0,0.035392,0.353504,10708.912308,0.225074,0.0,0.0
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,34,inf,0.0,0.0,0.064328,0.642529,57675.815873,1.212201,0.0,0.0
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,34,inf,0.0,0.0,0.056887,0.568201,27380.321715,0.575466,0.0,0.0
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,34,inf,0.0,0.0,0.056887,0.568201,27380.321715,0.575466,0.0,0.0
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,34,inf,0.0,0.0,0.056887,0.568201,27380.321715,0.575466,0.0,0.0


In [7]:

df_breakfaith.info()
df_breakfaith.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 4 columns):
EID          3657 non-null int64
TYPECODE     3657 non-null int64
FBDATE       3657 non-null object
SXENDDATE    114 non-null object
dtypes: int64(2), object(2)
memory usage: 114.4+ KB


,EID,TYPECODE,FBDATE,SXENDDATE
0,10751,1270862,2015/3/1,NaN
1,10751,1271420,2015/3/1,NaN
2,10751,66052879,2015/3/1,NaN
3,10751,66062576,2015/3/1,NaN
4,26642,289401,2014/8/1,NaN


In [8]:
df_breakfaith.loc[df_breakfaith['SXENDDATE'].isnull(),['SXENDDATE']] = '2018/1/1'

def time(x):
    y = x[:x.find('/')]
    m = int(x[x.find('/')+1:x.rfind('/')])
    if m < 10: m = '0'+str(m)
    else: m = str(m)
        
    return y + '-' + m

df_breakfaith['FBDATE'] = df_breakfaith['FBDATE'].apply(time)
df_breakfaith['SXENDDATE'] = df_breakfaith['SXENDDATE'].apply(time)


In [9]:
df_breakfaith = df_breakfaith.sort_values(['FBDATE','SXENDDATE'])

df_breakfaith.info()
df_breakfaith.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3657 entries, 1163 to 3636
Data columns (total 4 columns):
EID          3657 non-null int64
TYPECODE     3657 non-null int64
FBDATE       3657 non-null object
SXENDDATE    3657 non-null object
dtypes: int64(2), object(2)
memory usage: 142.9+ KB


,EID,TYPECODE,FBDATE,SXENDDATE
1163,494055,13428,2013-10,2015-07
2169,42215,30461,2013-10,2015-07
2171,42215,30460,2013-10,2015-07
3148,144321,10207,2013-10,2015-07
37,212895,11117,2013-10,2018-01


In [10]:
EIDS = set(df_breakfaith['EID'])

print(len(EIDS))

columns = df_breakfaith.columns
df_xbreakfaith = pd.DataFrame(columns=columns)

print(columns)


1303
Index(['EID', 'TYPECODE', 'FBDATE', 'SXENDDATE'], dtype='object')


In [11]:
k = 0
for EID in EIDS:
    if k%3000 == 0:
        print('第%d次处理--------->',k)
    k+=1
    tmp = df_breakfaith[df_breakfaith['EID'] == EID]
    row = [EID,tmp['TYPECODE'].values,tmp['FBDATE'].values,tmp['SXENDDATE'].values]
    
    df_xbreakfaith = df_xbreakfaith.append(pd.Series(row,columns),ignore_index=True)
    

第%d次处理---------> 0


In [12]:
df_xbreakfaith.info()
df_xbreakfaith.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 4 columns):
EID          1303 non-null float64
TYPECODE     1303 non-null object
FBDATE       1303 non-null object
SXENDDATE    1303 non-null object
dtypes: float64(1), object(3)
memory usage: 40.8+ KB


,EID,TYPECODE,FBDATE,SXENDDATE
0,346112.0,"[172536, 79640528, 310487]","[2014-06, 2014-06, 2014-08]","[2018-01, 2018-01, 2018-01]"
1,493575.0,"[1190868, 96509148]","[2015-02, 2015-02]","[2018-01, 2018-01]"
2,114696.0,"[1804463, 96486267]","[2015-06, 2015-06]","[2018-01, 2018-01]"
3,342025.0,"[174853, 41163579]","[2014-06, 2014-06]","[2018-01, 2018-01]"
4,376845.0,"[510385, 83560448]","[2014-09, 2014-09]","[2018-01, 2018-01]"


In [13]:
df_xbreakfaith['BREAKFAITH_CNT'] = df_xbreakfaith['TYPECODE'].apply(lambda x: len(x))

df_xbreakfaith['BREAKFAITH_FIRST_FIRST_TIME'] = df_xbreakfaith['FBDATE'].apply(lambda x: x[0])
df_xbreakfaith['BREAKFAITH_FIRST_END_TIME'] = df_xbreakfaith['FBDATE'].apply(lambda x: x[-1])
df_xbreakfaith['BREAKFAITH_END_FIRST_TIME'] = df_xbreakfaith['SXENDDATE'].apply(lambda x: x[0])



In [10]:
df_xbreakfaith.to_csv('../data/public/8breakfaith_1.csv',index=False,index_label=False)
df_xbreakfaith.columns



Index(['EID', 'TYPECODE', 'FBDATE', 'SXENDDATE', 'BREAKFAITH_CNT',
       'BREAKFAITH_FIRST_FIRST_TIME', 'BREAKFAITH_FIRST_END_TIME',
       'BREAKFAITH_END_FIRST_TIME'],
      dtype='object')

In [14]:
df_all = pd.merge(df_all,df_xbreakfaith[['EID','BREAKFAITH_CNT','BREAKFAITH_FIRST_FIRST_TIME', 'BREAKFAITH_FIRST_END_TIME',
                                       'BREAKFAITH_END_FIRST_TIME']],how='left',on=['EID'])


In [16]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Columns: 182 entries, EID to BREAKFAITH_END_FIRST_TIME
dtypes: float64(114), int64(48), object(20)
memory usage: 356.2+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,LAWSUIT_HY_CNT_AVG,LAWSUIT_HY_CNT_ALL_RATE,LAWSUIT_LAWAMOUNT_MEAN_HY_AVG,LAWSUIT_LAWAMOUNT_MEAN_ALL_RATE_y,LAWSUIT_CNT_HY_RATE,LAWSUIT_LAWAMOUNT_MEAN_HY_RATE,BREAKFAITH_CNT,BREAKFAITH_FIRST_FIRST_TIME,BREAKFAITH_FIRST_END_TIME,BREAKFAITH_END_FIRST_TIME
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,0.035392,0.353504,10708.912308,0.225074,0.0,0.0,NaN,NaN,NaN,NaN
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,0.064328,0.642529,57675.815873,1.212201,0.0,0.0,NaN,NaN,NaN,NaN
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,0.056887,0.568201,27380.321715,0.575466,0.0,0.0,NaN,NaN,NaN,NaN
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,0.056887,0.568201,27380.321715,0.575466,0.0,0.0,NaN,NaN,NaN,NaN
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,0.056887,0.568201,27380.321715,0.575466,0.0,0.0,NaN,NaN,NaN,NaN


In [17]:
# 所有有管时间的特征，用注册日期填充，其他的用0填充
df_all['BREAKFAITH_CNT'] = df_all['BREAKFAITH_CNT'].fillna(0)

modelist = [ 'BREAKFAITH_FIRST_FIRST_TIME', 'BREAKFAITH_FIRST_END_TIME','BREAKFAITH_END_FIRST_TIME']

for d in modelist:
    df_all.loc[df_all[d].isnull(),d] = df_all.loc[df_all[d].isnull(),'RGYEAR']



In [18]:
def timeDiff(x):
    a = x[:x.find(':')]
    b = x[x.find(':')+1:]
    y = int(a[:a.find('-')]) - int(b[:b.find('-')])
    m = int(a[a.find('-')+1:]) - int(b[b.find('-')+1:])
    return y * 12 + m


df_all['BREAKFAITH_FIRST_END_FIRST_DIFF'] = (df_all['BREAKFAITH_END_FIRST_TIME'] + ':' + df_all['BREAKFAITH_FIRST_FIRST_TIME']).apply(timeDiff)
df_all['BREAKFAITH_FIRST_RGYEAR_DIFF'] = (df_all['BREAKFAITH_FIRST_FIRST_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)
df_all['BREAKFAITH_END_RGYEAR_DIFF'] = (df_all['BREAKFAITH_FIRST_END_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)
df_all['BREAKFAITH_END_RGYEAR_DIFF'] = (df_all['BREAKFAITH_END_FIRST_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)


df_all['BREAKFAITH_FIRST_CHANGE_FIRST_DIFF'] = (df_all['BREAKFAITH_FIRST_FIRST_TIME'] + ':' + df_all['FIRST_CHANGE_TIME']).apply(timeDiff)
df_all['BREAKFAITH_END_CHANGE_FIRST_DIFF'] = (df_all['BREAKFAITH_FIRST_END_TIME'] + ':' + df_all['FIRST_CHANGE_TIME']).apply(timeDiff)
df_all['BREAKFAITH_FIRST_CHANGE_END_DIFF'] = (df_all['BREAKFAITH_FIRST_FIRST_TIME'] + ':' + df_all['END_CHANGE_TIME']).apply(timeDiff)
df_all['BREAKFAITH_END_CHANGE_END_DIFF'] = (df_all['BREAKFAITH_FIRST_END_TIME'] + ':' + df_all['END_CHANGE_TIME']).apply(timeDiff)



In [20]:
df_all['BREAKFAITH_PRE_MONTH_CNT'] = df_all['BREAKFAITH_END_RGYEAR_DIFF'] / df_all['BREAKFAITH_CNT']
df_all['BREAKFAITH_CNT_ALL_RATE'] = df_all['BREAKFAITH_CNT'] / df_all['BREAKFAITH_CNT'].mean()


In [21]:
tmp = pd.DataFrame()

tmp['BREAKFAITH_HY_CNT_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['BREAKFAITH_CNT'].mean()
tmp['BREAKFAITH_HY_CNT_ALL_RATE'] = tmp['BREAKFAITH_HY_CNT_AVG'] / df_all['BREAKFAITH_CNT'].mean()


tmp['HY'] = tmp.index

df_all = pd.merge(df_all,tmp,how='left',on=['HY'])


In [22]:
df_all['BREAKFAITH_CNT_HY_RATE'] = df_all['BREAKFAITH_CNT'] / df_all['BREAKFAITH_HY_CNT_AVG']

In [23]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Columns: 194 entries, EID to BREAKFAITH_CNT_HY_RATE
dtypes: float64(119), int64(55), object(20)
memory usage: 379.6+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,BREAKFAITH_END_RGYEAR_DIFF,BREAKFAITH_FIRST_CHANGE_FIRST_DIFF,BREAKFAITH_END_CHANGE_FIRST_DIFF,BREAKFAITH_FIRST_CHANGE_END_DIFF,BREAKFAITH_END_CHANGE_END_DIFF,BREAKFAITH_PRE_MONTH_CNT,BREAKFAITH_CNT_ALL_RATE,BREAKFAITH_HY_CNT_AVG,BREAKFAITH_HY_CNT_ALL_RATE,BREAKFAITH_CNT_HY_RATE
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,0,-182,-182,-182,-182,NaN,0.0,0.007324,0.510965,0.0
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,0,-146,-146,-146,-146,NaN,0.0,0.016958,1.183042,0.0
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,0,-26,-26,-26,-26,NaN,0.0,0.009002,0.628041,0.0
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,0,-14,-14,-14,-14,NaN,0.0,0.009002,0.628041,0.0
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,0,-86,-86,-86,-86,NaN,0.0,0.009002,0.628041,0.0


In [24]:
# 得到在df_xchange的所有数据，
df_all = df_all.fillna(0)
df_all.to_csv('../data/alldata/df_data12345678.csv',index=False,index_label=False)

